# Setup
### Imports

In [1]:
import sys
sys.path.append('../')
del sys
%reload_ext autoreload
%autoreload 2

import modeling.models as models
from toolbox.utils import to_class_name, load_task, get_pretrained_model
from toolbox.parameters import SCORES_NAMES, MODELS_RANDOM_SEED
from toolbox.paths import MODELING_TASK_FOR_BASELINES_PATH, PRETRAINED_MODELS_PATH
from run_baseline import play_baseline

### Parameters

In [10]:
task_name = "context_dependent_same_type"
experiment_name = 'test_1_1'
short = False
root = "../"

### Load the data

In [4]:
task = load_task(task_name=task_name,
                 folder_path=MODELING_TASK_FOR_BASELINES_PATH,
                 short=short,
                 root=root)

Task loaded from ../results/modeling_task/baselines_split/context_dependent_same_type.pkl.



# Run the baselines
## Basic baselines
### Models

In [11]:
model_names = [
    "random",
    "frequency",
    #"summaries_count",
    #"summaries_unique_count",
    #"summaries_overlap",
    #"activated_summaries",
    #"context_count",
    #"context_unique_count",
    #"summaries_context_count",
    #"summaries_context_unique_count",
    #"summaries_context_overlap",
]
model_names = [to_class_name(model_name) for model_name in model_names]

### Run the models

In [12]:
for model_name in model_names:
    print(model_name)
    
    model = getattr(models, model_name)(scores_names=SCORES_NAMES,
                                        relevance_level=task.relevance_level,
                                        experiment_name=experiment_name,
                                        pretrained_model=None,
                                        pretrained_model_dim=None,
                                        random_seed=MODELS_RANDOM_SEED)
    
    play_baseline(task=task,
                  model=model)
    
    model.display_metrics_valid()

Random
Validation of the model...



NotImplementedError: Got <class 'NoneType'>, but numpy array, torch tensor, or caffe2 blob name are expected.

## Embedding baselines
### Models

In [7]:
model_names = [
    "summaries_average_embedding",
    "summaries_overlap_average_embedding",
    "context_average_embedding",
    "summaries_context_average_embedding",
    "summaries_context_overlap_average_embedding",
]
model_names = [to_class_name(model_name) for model_name in model_names]

### Pretrained model

In [8]:
word_embedding, word_embedding_dim = get_pretrained_model(pretrained_model_name="word2vec",
                                                          folder_path=PRETRAINED_MODELS_PATH,
                                                          root=root)

Word2Vec embedding loaded.



### Run the models

In [9]:
for model_name in model_names:
    print(model_name)
    
    model = getattr(models, model_name)(scores_names=SCORES_NAMES,
                                        relevance_level=task.relevance_level,
                                        experiment_name=experiment_name,
                                        pretrained_model=word_embedding,
                                        pretrained_model_dim=word_embedding_dim,
                                        random_seed=MODELS_RANDOM_SEED)
    
    play_baseline(task=task,
                  model=model)
    
    model.display_metrics_valid()

SummariesAverageEmbedding
Validation of the model...




Validation Score: 0.07822
Scores evaluated on the validation set:
average_precision: 0.07822
recall_at_10: 0.12232
reciprocal_best_rank: 0.13005
reciprocal_average_rank: 0.02644
ndcg_at_10: 0.09232


SummariesOverlapAverageEmbedding
Validation of the model...




Validation Score: 0.10471
Scores evaluated on the validation set:
average_precision: 0.10471
recall_at_10: 0.16065
reciprocal_best_rank: 0.17590
reciprocal_average_rank: 0.03437
ndcg_at_10: 0.12572


ContextAverageEmbedding
Validation of the model...




Validation Score: 0.06141
Scores evaluated on the validation set:
average_precision: 0.06141
recall_at_10: 0.09377
reciprocal_best_rank: 0.10511
reciprocal_average_rank: 0.01819
ndcg_at_10: 0.07022


SummariesContextAverageEmbedding
Validation of the model...




Validation Score: 0.07925
Scores evaluated on the validation set:
average_precision: 0.07925
recall_at_10: 0.13016
reciprocal_best_rank: 0.13118
reciprocal_average_rank: 0.02653
ndcg_at_10: 0.09486


SummariesContextOverlapAverageEmbedding


AttributeError: module 'modeling.models' has no attribute 'SummariesContextOverlapAverageEmbedding'

## BART baselines
### Models

In [ ]:
model_names = [
    "summaries_bart_mnli",
]
model_names = [to_class_name(model_name) for model_name in model_names]

### Pretrained model

In [13]:
bart_mnli, _ = get_pretrained_model(pretrained_model_name="bart_mnli",
                                    folder_path=PRETRAINED_MODELS_PATH,
                                    root=root)

loading archive file ../modeling/pretrained_models/bart.large.mnli
| dictionary: 50264 types
Registering classification head: mnli
Pretrained BART.mnli loaded.



### Run the models

In [ ]:
for model_name in model_names:
    print(model_name)
    
    model = getattr(models, model_name)(scores_names=SCORES_NAMES,
                                        relevance_level=task.relevance_level,
                                        experiment_name=experiment_name,
                                        pretrained_model=bart_mnli,
                                        pretrained_model_dim=None,
                                        random_seed=MODELS_RANDOM_SEED)
    
    play_baseline(task=task,
                  model=model)
    
    model.display_metrics_valid()

SummariesAverageEmbedding
Validation of the model...




Validation Score: 0.07634
Scores evaluated on the validation set:
average_precision: 0.07634
recall_at_10: 0.11513
reciprocal_best_rank: 0.13253
reciprocal_average_rank: 0.02128
ndcg_at_10: 0.09071


SummariesOverlapAverageEmbedding
Validation of the model...




Validation Score: 0.10399
Scores evaluated on the validation set:
average_precision: 0.10399
recall_at_10: 0.15489
reciprocal_best_rank: 0.19313
reciprocal_average_rank: 0.02827
ndcg_at_10: 0.12905


ContextAverageEmbedding
Validation of the model...

